# Show whether the analytic Jacobians are correct by testing against finite-difference calculations

In [ ]:
cd("../")

In [ ]:
# Let us observe the progress during the RT computations
ENV["XRTM_PROGRESS"] = "1"

# For good font rendering, we suggest installing the "JuliaMono" font
# which provides full support for the various Unicode glyphs.
ENV["GKS_FONTPATH"] = "~/Library/Fonts/"

In [ ]:
# Load these modules ahead of running the application
using Plots, LaTeXStrings
gr()

In [ ]:
Plots.default()

Plots.default(
    fontfamily = "JuliaMono-Regular",
    titlefont = (10, "JuliaMono-Regular"),
    legendfont = (8, "JuliaMono-Regular"),
    guidefont = (8, "JuliaMono-Regular", :black),
    tickfont = (8, "JuliaMono-Regular", :black),
    rightmargin=5Plots.mm,
    leftmargin=5Plots.mm,
    bottommargin=5Plots.mm
)

In [ ]:
# Define command-line arguments

# This lets us run with 0 iterations, i.e. just set up, but no forward model run

my_args = [
    "--solar_model", "./example_data/l2_solar_model.h5",
    "--L1b", "./example_data/2021030111564431_inputs.h5",
    "--L2Met", "./example_data/2021030111564431_inputs.h5",
    "--L2CPr", "./example_data/2021030111564431_inputs.h5",
    "--sounding_id", "2021030111564431",
    "--spec", "1",
    "--polarized", "true",
    "--aerosols", "true",
    "--retrieve_aerosols", "true",
    "--LSI", "true",
    "--o2_scale", "1.0048",
    "--co2_scale_weak", "0.994",
    "--co2_scale_strong", "0.998",
    "--gamma", "1000.0",
    "--dsigma_scale", "2.0",
    "--max_iterations", "0",
    "--output", "2021030111564431.h5",
]

# Get rid of existing command line arguments
empty!(ARGS)

for a in my_args
    push!(ARGS, a)
end

# Run the retrieval and return the Buffer and Solver objects
buf, solver, fm_kwargs = include("./run.jl");

## First run: derivatives enabled

In [ ]:
# Add "calc_derivs" if needed
for swin in buf.spectral_window
    rt = buf.rt[swin] # Grab the RT object

    for mo in rt.model_options
        if !("calc_derivs" in mo["options"])
           push!(mo["options"], "calc_derivs")
        end
    end
    
end

ENV["XRTM_PROGRESS"] = "1";
# Run the forward model
@time solver.forward_model(solver.state_vector; fm_kwargs...)

# Note - this DOES NOT update the state vector; the atmosphere can be mutated, but a correct forward model will restore them to the initial state

In [ ]:
# Make some number of iterations
ENV["XRTM_PROGRESS"] = "1";

@time next_iteration!(solver; fm_kwargs)
@time next_iteration!(solver; fm_kwargs)

#@time solver.forward_model(solver.state_vector; fm_kwargs...)
calculate_chi2(solver)


In [ ]:
# Store the results from the analytic computation of the Jacobian K, the forward model result is also needed
analytic_K = RE.create_K_from_solver(solver);
analytic_fm = RE.get_modeled(solver);

In [ ]:
logger = ConsoleLogger(stderr, Logging.Error);
global_logger(logger);

In [ ]:
# To remove the "calc_deriv" option that tells XRTM to calculate derivatives.
# We don't need derivatives for the next step, and it makes the forward model run significantly faster.

for swin in buf.spectral_window
    rt = buf.rt[swin] # Grab the RT object

    for mo in rt.model_options
        filter!(!=("calc_derivs"), mo["options"])
    end
    
end

# Let's also make logging less verbose because we don't want to fill up the screen
logger = ConsoleLogger(stderr, Logging.Error);
global_logger(logger);

# And remove the XRTM progress
ENV["XRTM_PROGRESS"] = "0";

In [ ]:
SV = solver.state_vector
perturbations = zeros(length(SV)) * NaN

for (idx, sve) in enumerate(SV.state_vector_elements)

    
    if sve isa RE.SurfacePressureSVE
        perturbations[idx] = ustrip(sve.unit, 1.0u"Pa")
    end
    
    if sve isa RE.TemperatureOffsetSVE
        perturbations[idx] = 0.1 # this should be in K
    end
    
    if sve isa RE.GasVMRProfileSVE
    #    perturbations[idx] = 1.0 # this should be in ppm
    end

    if sve isa RE.GasLevelScalingFactorSVE
        perturbations[idx] = 0.01
    end
    
    if sve isa RE.DispersionPolynomialSVE
        perturbations[idx] = 0.000001 * sve.iterations[end]
    end
    
    if sve isa RE.BRDFPolynomialSVE
        # Take some fraction of current value
        if sve.coefficient_order == 0
            perturbations[idx] = 0.01 * sve.iterations[end]
        else
            perturbations[idx] = 0.01
        end
    end 
    
    if sve isa RE.AerosolWidthSVE
        # Makes aerosols slightly wider
        perturbations[idx] = 0.00001 * sve.iterations[end]
    end

    if sve isa RE.AerosolHeightSVE
        # Moves aerosols closer to the surface..
        perturbations[idx] = 0.00001 * sve.iterations[end] 
    end
    
    if sve isa RE.AerosolOpticalDepthSVE
        # These are in log-space! Make them optically denser
        perturbations[idx] = 0.00001
    end

end

In [ ]:
fd_K = similar(analytic_K);
fd_K[:] .= NaN;

for idx in 1:length(SV)

    # Skip SVEs that have no valid perturbations
    isnan(perturbations[idx]) && continue
    
    println("Evaluating $(idx) for $(SV.state_vector_elements[idx])")
    # Add a "fake" iteration to that state vector, which contains our new perturbed state
    for sve in SV.state_vector_elements
        # Simply repeat the current value
        push!(sve.iterations, sve.iterations[end])
    end
    
    # .. but replace the value for the SVE that we are currently investigating
    SV.state_vector_elements[idx].iterations[end] += perturbations[idx]
    
    # Evaluate the forward model    
    @time solver.forward_model(solver.state_vector; fm_kwargs...);
    
    # The approximate Jacobian is then {F(x + Δx) - F(x)} / Δx
    perturbed_fm = RE.get_modeled(solver);
    @. fd_K[:,idx] = (perturbed_fm[:] - analytic_fm[:]) / perturbations[idx]
    
    # Remove the fake iteration!
    for sve in SV.state_vector_elements
        pop!(sve.iterations)
    end

    #break
end

In [ ]:
for idx in 1:length(perturbations)

    isnan(perturbations[idx]) && continue

    p1 = Plots.plot(
        fd_K[:,idx], 
        label="Δx = $(perturbations[idx]) [$(solver.state_vector.state_vector_elements[idx].unit)]",
        linewidth=2., size=(800, 500)
    )
    Plots.plot!(analytic_K[:,idx], label="Analytic", linestyle=:dash)
    Plots.title!("$(SV.state_vector_elements[idx])")

    resid = @. (analytic_K[:,idx] - fd_K[:,idx])
    #resid_rel = @. 100 * (analytic_K[:,idx] - fd_K[:,idx]) / maximum(abs.(fd_K[:,idx]))

    rrms = sqrt(mean(resid .^ 2))

    p2 = Plots.plot(resid, label="Difference (analytic - FD)", linewidth=1., size=(800, 500))
    Plots.title!(@sprintf "RMS = %.4e " rrms)

    disp_plot = plot(p1, p2, layout=(2, 1))
    display(disp_plot)

end